In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
os.chdir('drive/My Drive/DataDays/second_round')

In [0]:
df = pd.read_csv('divar_posts_dataset.csv')

In [0]:
df['cat1_2'] = df.cat1.astype(str).str.cat(df.cat2.astype(str), sep='|')
df['cat1_2_3'] = df.cat1_2.astype(str).str.cat(df.cat3.astype(str), sep='|')

In [6]:
import sys
!{sys.executable} -m pip install hazm

In [0]:
import hazm
import nltk
from bs4 import BeautifulSoup
import re
from keras.preprocessing.text import Tokenizer,text_to_word_sequence


In [0]:
train,test = train_test_split(df,random_state = 100)

In [0]:
MAX_SENT_LENGTH = 100
MAX_SENTS = 15
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [13]:
texts =[]
r = []

for index,row in train.iterrows():
    text = row['title'] + ' ' + row['desc']
    text = text.replace(u'\xa0', u' ')
    texts.append(text)
    
    sentences = hazm.sent_tokenize(text)
    r.append(sentences)
    
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)

data_train = np.zeros((len(texts), MAX_SENTS, MAX_SENT_LENGTH), dtype='int32')

for i, sentences in enumerate(r):
    for j, sent in enumerate(sentences):
        if j< MAX_SENTS:
            wordTokens = text_to_word_sequence(sent)
            k=0
            for _, word in enumerate(wordTokens):
                if k<MAX_SENT_LENGTH and tokenizer.word_index[str(word)]<MAX_NB_WORDS:
                    data_train[i,j,k] = tokenizer.word_index[word]
                    k=k+1
              
word_index_train = tokenizer.word_index
print('No. of %s unique tokens.' % len(word_index_train))

No. of 437565 unique tokens.


In [16]:
texts =[]
r = []

for index,row in test.iterrows():
    text = row['title'] + ' ' + row['desc']
    text = text.replace(u'\xa0', u' ')
    texts.append(text)
    
    sentences = hazm.sent_tokenize(text)
    r.append(sentences)
    
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)

data_test = np.zeros((len(texts), MAX_SENTS, MAX_SENT_LENGTH), dtype='int32')

for i, sentences in enumerate(r):
    for j, sent in enumerate(sentences):
        if j< MAX_SENTS:
            wordTokens = text_to_word_sequence(sent)
            k=0
            for _, word in enumerate(wordTokens):
                if k<MAX_SENT_LENGTH and tokenizer.word_index[str(word)]<MAX_NB_WORDS:
                    data_test[i,j,k] = tokenizer.word_index[word]
                    k=k+1
      
word_index_test = tokenizer.word_index
print('No. of %s unique tokens.' % len(word_index_test))

No. of 199924 unique tokens.


In [0]:
labels = {j:i for i,j in enumerate(train.cat1_2_3.unique())}

In [0]:
X_train = data_train
Y_train = train.apply(lambda i : labels[i.cat1_2_3], axis=1)

In [0]:
X_test = data_test
Y_test = test.apply(lambda i : labels[i.cat1_2_3], axis=1)

In [20]:


Y_train = to_categorical(np.asarray(Y_train))
Y_test = to_categorical(np.asarray(Y_test))

print('Shape of label tensor:', Y_train.shape)
print('Shape of label tensor:', Y_test.shape)


Shape of label tensor: (710726, 87)
Shape of label tensor: (236909, 87)


In [21]:
Y_train

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [0]:
from imblearn.keras import balanced_batch_generator
from imblearn.under_sampling import NearMiss

In [26]:
from keras.layers import Dense, Input, Flatten,Bidirectional,TimeDistributed
from keras.layers import GlobalAveragePooling1D, Embedding , Reshape,Concatenate
from keras.models import Model
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
EMBEDDING_DIM = 100 



embedding_layer = Embedding(len(word_index_train) + len(word_index_test) + 1,
                            EMBEDDING_DIM,
                            input_length=MAX_SENT_LENGTH,
                            trainable=True)

sentence_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sentence_input)
l_lstm = Bidirectional(LSTM(100))(embedded_sequences)
sentEncoder = Model(sentence_input, l_lstm)

inputs = Input(shape=(MAX_SENTS,MAX_SENT_LENGTH), dtype='int32')
encoder = TimeDistributed(sentEncoder)(inputs)
l_lstm_sent = Bidirectional(LSTM(100))(encoder)
preds = Dense(87, activation='softmax')(l_lstm_sent)
model = Model(inputs, preds)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 15, 100)           0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 15, 200)           63909800  
_________________________________________________________________
bidirectional_3 (Bidirection (None, 200)               240800    
_________________________________________________________________
dense_1 (Dense)              (None, 87)                17487     
Total params: 64,168,087
Trainable params: 64,168,087
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.fit(X_train, Y_train, 
                        batch_size= 256,
                        epochs=10,
                        validation_data=(X_test,Y_test),
                        shuffle=True)

Instructions for updating:
Use tf.cast instead.
Train on 710726 samples, validate on 236909 samples
Epoch 1/10
710726/710726 [==============================] - 3726s 5ms/step - loss: 0.8474 - acc: 0.7976 - val_loss: 5.7543 - val_acc: 0.1437
Epoch 2/10
710726/710726 [==============================] - 3709s 5ms/step - loss: 0.3741 - acc: 0.8999 - val_loss: 5.9636 - val_acc: 0.1419
Epoch 3/10
710726/710726 [==============================] - 3731s 5ms/step - loss: 0.3138 - acc: 0.9118 - val_loss: 6.2611 - val_acc: 0.1273
Epoch 4/10
710726/710726 [==============================] - 3737s 5ms/step - loss: 0.2779 - acc: 0.9197 - val_loss: 6.4961 - val_acc: 0.1420
Epoch 5/10
710726/710726 [==============================] - 3715s 5ms/step - loss: 0.2499 - acc: 0.9266 - val_loss: 6.7480 - val_acc: 0.1437
Epoch 6/10
710726/710726 [==============================] - 3737s 5ms/step - loss: 0.2264 - acc: 0.9328 - val_loss: 7.0511 - val_acc: 0.1406
Epoch 7/10
710726/710726 [============================

In [0]:
from sklearn.metrics import roc_auc_score
output_test = model.predict(X_test)


In [0]:
from sklearn.metrics import classification_report
Y_test = np.argmax(Y_test,axis=1)
output_test = np.argmax(output_test,axis=1)
print(classification_report(Y_test, output_test))

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.80      0.31      0.44       960
           1       0.88      0.29      0.43     13356
           2       0.54      0.71      0.62      5687
           3       0.51      0.48      0.49      7760
           4       0.24      0.31      0.27      1892
           5       0.95      0.25      0.39      8277
           6       0.49      0.78      0.60      1308
           7       0.35      0.25      0.29       829
           8       0.98      0.11      0.19     15507
           9       0.65      0.02      0.03      6085
          10       0.00      0.00      0.00      1125
          11       0.97      0.04      0.08     30281
          12       0.85      0.07      0.12      1563
          13       1.00      0.02      0.03      8010
          14       0.33      0.83      0.48       360
          15       0.61      0.05      0.10      2841
          16       0.81      0.01      0.03     13951
          17       0.99    

In [0]:
tst = pd.read_csv('dataset.csv')

In [0]:
X_tst = tst.apply(lambda i : str(i.title) + ' ' + str(i.desc), axis=1)


In [0]:
texts_tst = X_tst.astype(str)


In [0]:
sequences_tst = tokenizer.texts_to_sequences(texts_tst)


In [0]:
MAX_SEQUENCE_LENGTH = 150

x_tst = pad_sequences(sequences_tst, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data test tensor:', x_tst.shape)

Shape of data test tensor: (200000, 150)


In [0]:
output_test = model.predict(x_tst)


In [0]:
pred=np.argmax(output_test,axis=1)

In [0]:
tst['cat1_2_3']=pred

In [0]:
merged = {}
for key, value in labels.items():
    merged[value] = key


In [0]:
pred.shape


(200000,)

In [0]:
tst['cat1_2_3'] = tst['cat1_2_3'].map(merged)

In [0]:
cat_tmp = tst['cat1_2_3'].str.split("|",n=1,expand=True)
tst['cat1'] = cat_tmp[0]
tst['cat2_3']=cat_tmp[1] 
cat2_tmp = tst['cat2_3'].str.split("|",n=1,expand=True)
tst['cat2'] = cat2_tmp[0]
tst['cat3']=cat2_tmp[1] 

In [0]:
tst[150000:150100]

,Unnamed: 0,archive_by_user,city,created_at,desc,id,image_count,platform,price,title,cat1_2_3,cat1,cat2_3,cat2,cat3
150000,150000,False,Tehran,Saturday 07PM,دوچرخه کره‌ای 28,2452480101955,0,web,500000,دوچرخه 28 کره ای,leisure-hobbies|bicycle|nan,leisure-hobbies,bicycle|nan,bicycle,nan
150001,150001,True,Isfahan,Saturday 08AM,میز تلوزیون چوبی با صفحه گردون بسیار تمیز و سا...,51060292769512,2,mobile,40000,میز تلوزیون چوبی سالم و تمیز و مناسب,for-the-home|furniture-and-home-decore|tv-and-...,for-the-home,furniture-and-home-decore|tv-and-stereo-furniture,furniture-and-home-decore,tv-and-stereo-furniture
150002,150002,True,Ahvaz,Friday 03AM,دستمال پنج تکه قیمت فوق العاده مناسب‌. برای مش...,56865502995784,1,mobile,4000,دستمال آشپزخانه,for-the-home|furniture-and-home-decore|antique...,for-the-home,furniture-and-home-decore|antiques-and-art,furniture-and-home-decore,antiques-and-art
150003,150003,True,Tehran,Saturday 11AM,مدل ۸۸\r\r\r\nبیمه تا اردیبهشت ۹۶\r\r\r\n۲حلقه...,58314610442706,4,mobile,-1,خودرو پیکان وانت ۸۸,vehicles|cars|light,vehicles,cars|light,cars,light
150004,150004,True,Tehran,Saturday 12PM,بسیار شیک و در حد نو ، فقط دو بار پوشیده،،سایز...,48970600177323,1,mobile,35000,کفش کتونی سایز 24,personal|childrens-clothing-and-shoe|nan,personal,childrens-clothing-and-shoe|nan,childrens-clothing-and-shoe,nan
150005,150005,True,Tehran,Wednesday 08AM,تحویل درب کارخانه رنگ مشكی,7058729284970,0,mobile,88500000,جک s5 دنده ای,vehicles|cars|light,vehicles,cars|light,cars,light
150006,150006,True,Isfahan,Thursday 05PM,یه کفتر نر زینتی خیلی مسته من تو کار زینتی نیس...,40987664090538,2,mobile,35000,کبوتر نر ساتن زیبا,leisure-hobbies|animals|birds,leisure-hobbies,animals|birds,animals,birds
150007,150007,True,Tehran,Thursday 09PM,فقط تماس لطفا.\r\r\r\nاس ندید پاسخگو نیستم.\r\...,63722024387305,2,mobile,50000,ضبط فابریک ۲۰۶,vehicles|parts-accessories|nan,vehicles,parts-accessories|nan,parts-accessories,nan
150008,150008,False,Tehran,Sunday 01PM,قسمت های سفید تماما ویو وارداتی بقیه قسمت ها د...,32547584323596,4,mobile,1850000,"سرویس خواب تک نفره ""مارلین""",for-the-home|furniture-and-home-decore|beds-be...,for-the-home,furniture-and-home-decore|beds-bedroom,furniture-and-home-decore,beds-bedroom
150009,150009,True,Mashhad,Saturday 11AM,یک گوشی ال جی مدل m3 طرحه\r\r\r\nسالم سالم \r...,62394333839571,2,mobile,160000,گوشی ال جی m3,electronic-devices|mobile-tablet|mobile-phones,electronic-devices,mobile-tablet|mobile-phones,mobile-tablet,mobile-phones


In [0]:
tst = tst.drop(columns=['Unnamed: 0', 'archive_by_user', 'city', 'created_at', 'desc', 'id',
       'image_count', 'platform', 'price', 'title', 'cat1_2_3',
       'cat2_3',],axis=1)


In [0]:
tst.replace(['nan'], '', inplace=True)

In [0]:
tst

,cat1,cat2,cat3
0,for-the-home,furniture-and-home-decore,tables-and-chairs
1,electronic-devices,mobile-tablet,tablet
2,for-the-home,furniture-and-home-decore,tv-and-stereo-furniture
3,for-the-home,utility,instrument-cleaning-tailoring
4,vehicles,cars,light
5,leisure-hobbies,animals,birds
6,personal,health-beauty,
7,electronic-devices,game-consoles-and-video-games,
8,leisure-hobbies,hobby-collectibles,historical-objects
9,vehicles,cars,light


In [0]:
tst.to_csv("tst_deep.csv")